In [ ]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=0100118819949233148f1287155d015505d4c854443898e4be21067b3ba94cca
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


In [ ]:
import wikipediaapi

# Initialize Wikipedia API with a custom user agent
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='CBOWModel/1.0 (https://yourdomain.com; contact@example.com)'
)

# Function to extract text from a Wikipedia page
def get_wikipedia_content(page_title):
    page = wiki_wiki.page(page_title)
    if page.exists():
        return page.text
    else:
        print(f"Page '{page_title}' not found.")
        return ""

# List of relevant pages for states and capitals
pages = ["List of U.S. state capitals", "Geography of the United States"]

# Collect the content into one corpus
corpus = ""
for page in pages:
    corpus += get_wikipedia_content(page) + "\n"

# Save the corpus to a text file
with open("us_states_capitals_corpus.txt", "w") as f:
    f.write(corpus)

print("Corpus saved successfully!")



Corpus saved successfully!


In [ ]:
from random import sample  # Ensure this is imported

# Load your custom Wikipedia text corpus
def load_custom_corpus(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        corpus = f.read()
    return corpus

# Use your corpus instead of loading from datasets
corpus_file = "us_states_capitals_corpus.txt"
corpus = load_custom_corpus(corpus_file)  # Corrected: Passing the variable

# Optional: Split the corpus into smaller chunks if needed
def sample_corpus(corpus, num_samples=5000):
    lines = corpus.split("\n")
    return "\n".join(sample(lines, min(num_samples, len(lines))))

# Get a trimmed version of the corpus with 5000 lines (or fewer if the corpus is smaller)
trimmed_corpus = sample_corpus(corpus, num_samples=5000)

# Confirm the trimmed corpus length
print(f"Trimmed Corpus Length: {len(trimmed_corpus.splitlines())} lines")

# Now you can proceed to use `trimmed_corpus` in your CBOW training or any other tasks






Trimmed Corpus Length: 276 lines


In [ ]:
import re
from collections import Counter

# 1. Load the corpus (assuming 'trimmed_corpus' has the preloaded content)
text = trimmed_corpus.lower()  # Convert text to lowercase

# 2. Remove special characters, punctuation, and digits using regex
text = re.sub(r'[^a-z\s]', '', text)  # Keep only letters and spaces

# 3. Tokenize the text into words (split by whitespace)
tokens = text.split()

# 4. Build vocabulary with word-to-index and index-to-word mappings
vocab = Counter(tokens)  # Count frequency of each word

# Optional: Limit vocabulary size (if needed) to most common words
# vocab = vocab.most_common(10000)  # Uncomment this line if you want a capped vocab size

vocab_size = len(vocab)

# 5. Create word-to-index and index-to-word dictionaries
word_to_index = {word: idx for idx, (word, _) in enumerate(vocab.items())}
index_to_word = {idx: word for word, idx in word_to_index.items()}

# 6. Print vocabulary size and sample of the mappings
print(f"Vocabulary Size: {vocab_size}")
print("Sample word-to-index mappings:", list(word_to_index.items())[:10])


Vocabulary Size: 1643
Sample word-to-index mappings: [('hilo', 0), ('references', 1), ('an', 2), ('insular', 3), ('area', 4), ('is', 5), ('a', 6), ('united', 7), ('states', 8), ('territory', 9)]


In [ ]:
import re
from collections import Counter

def vocab_frequency(text):
    """
    Creates a dictionary of word frequencies based on a dataset.

    Arguments
    ---------
    text : String
        Preprocessed text in which words will be counted.

    Returns
    -------
    vocab_dict : dictionary
        Dictionary of words and their frequencies with the format {word: frequency}.
    """
    vocab_dict = {}

    # Split the text into words
    words = text.split()

    # Count the frequency of each word
    for word in words:
        if word in vocab_dict:
            vocab_dict[word] += 1
        else:
            vocab_dict[word] = 1

    return vocab_dict

# Step 1: Preprocess and generate the vocabulary frequency dictionary from trimmed corpus
# Lowercase and clean the corpus to keep only alphabets and spaces
cleaned_text = re.sub(r'[^a-z\s]', '', trimmed_corpus.lower())

# Generate the vocabulary dictionary
vocabulary = vocab_frequency(cleaned_text)

# Step 2: Print vocabulary size
print(f"Vocabulary Size: {len(vocabulary)}")

# Step 3: Print top 10 most common words
sorted_vocab = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
print("Top 10 most common words and their frequencies:", sorted_vocab[:10])

# Step 4: Optional - Check the frequency of specific words
print(f"Frequency of 'state': {vocabulary.get('state', 0)}")
print(f"Frequency of 'capital': {vocabulary.get('capital', 0)}")


Vocabulary Size: 1643
Top 10 most common words and their frequencies: [('the', 786), ('of', 361), ('and', 244), ('in', 207), ('to', 112), ('states', 108), ('a', 101), ('united', 82), ('is', 73), ('as', 73)]
Frequency of 'state': 47
Frequency of 'capital': 47


In [ ]:
import torch
import torch.nn as nn
from collections import Counter

# Step 1: Generate the vocabulary from your corpus text
# Assuming 'text' contains the trimmed corpus
tokens = text.split()
vocab = Counter(tokens)

# Step 2: Create word-to-index mapping
def create_word_to_index(vocabulary):
    """
    Generates a word-to-index mapping from the vocabulary.

    Arguments
    ---------
    vocabulary : dict
        Dictionary containing words and their frequencies.

    Returns
    -------
    word_to_index : dict
        Mapping of words to indices.
    """
    word_to_idx = {'<OOV>': 0}  # Reserve 0 for OOV words
    for idx, word in enumerate(vocabulary.keys(), start=1):
        word_to_idx[word] = idx
    return word_to_idx

# Step 3: Generate the word-to-index mapping
word_to_index_mapping = create_word_to_index(vocab)

# Step 4: Verify the generated mapping
print("Sample of word-to-index mapping:")
for word, index in list(word_to_index_mapping.items())[:10]:
    print(f"{word}: {index}")

# Step 5: Initialize your CBOW model with the correct vocabulary size
vocab_size = len(word_to_index_mapping)  # Use the correct length of the mapping
model = CBOW(vocab_size).to(device)  # Move the model to the appropriate device
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



Sample of word-to-index mapping:
<OOV>: 0
hilo,: 1
1796–1803: 2
references: 3
an: 4
insular: 5
area: 6
is: 7
a: 8
united: 9


In [ ]:
import torch

def generate_dataset(data, window_size, word_to_index):
    """
    Method to generate training dataset for CBOW.

    Arguments
    ---------
    data : String
        Training dataset (a single string).
    window_size : int
        Size of the context window (number of words before and after the target).
    word_to_index : Dictionary
        Dictionary mapping words to indices with format {word: index}.

    Returns
    -------
    surroundings : Tensor
        Tensor with indices of surrounding words, shape (N x 2*window_size), where N is the number of samples.
    targets : Tensor
        Tensor with indices of target words, shape (N,).
    """

    surroundings = []
    targets = []

    # Split the input data into a list of words
    data = data.split()

    # Iterate through the dataset starting at window_size and stopping window_size before the end
    for i in range(window_size, len(data) - window_size):
        # Collect the surrounding words, window_size on both sides of the target word
        surrounding = data[i - window_size:i] + data[i + 1:i + 1 + window_size]

        # Convert the surrounding words to their indices (use <OOV> if word not found)
        surrounding_indices = [word_to_index.get(word, word_to_index['<OOV>']) for word in surrounding]

        # Get the target word (in the middle) and convert it to its index
        target_word = data[i]
        target_index = word_to_index.get(target_word, word_to_index['<OOV>'])

        # Append the surrounding indices and target index to their respective lists
        surroundings.append(surrounding_indices)
        targets.append(target_index)

    # Convert lists to tensors for PyTorch compatibility
    surroundings_tensor = torch.tensor(surroundings, dtype=torch.long)
    targets_tensor = torch.tensor(targets, dtype=torch.long)

    return surroundings_tensor, targets_tensor

# Ensure the preprocessed text is being used
text = trimmed_corpus.lower()  # Use the trimmed, cleaned corpus

# Generate the dataset using window size of 2
surroundings, targets = generate_dataset(text, window_size=2, word_to_index=word_to_index_mapping)

# Display the shapes of the tensors
print(f"Surroundings Tensor Shape: {surroundings.shape}")
print(f"Targets Tensor Shape: {targets.shape}")

# Display a sample of the data
print("Sample data:")
print(f"Surroundings (first 5): {surroundings[:5]}")
print(f"Targets (first 5): {targets[:5]}")


Surroundings Tensor Shape: torch.Size([7487, 4])
Targets Tensor Shape: torch.Size([7487])
Sample data:
Surroundings (first 5): tensor([[1, 2, 4, 5],
        [2, 3, 5, 6],
        [3, 4, 6, 7],
        [4, 5, 7, 8],
        [5, 6, 8, 9]])
Targets (first 5): tensor([3, 4, 5, 6, 7])


In [ ]:
import torch
import torch.nn as nn

class CBOW(nn.Module):
    def __init__(self, vocab_size, embed_dim=300):
        """
        Class to define the CBOW model.

        Attributes
        ----------
        vocab_size : int
            Size of the vocabulary.
        embed_dim : int
            Size of the embedding layer (dimension of word vectors).
        """
        super(CBOW, self).__init__()

        # Embedding layer: maps word indices to dense vectors of size embed_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Linear layer: projects averaged embedding to the vocabulary space
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        """
        Forward pass through the network.

        Arguments
        ---------
        x : Tensor
            Input tensor containing indices of the surrounding words.
            Shape: (batch_size, context_window * 2)

        Returns
        -------
        out : Tensor
            Output tensor representing the predicted scores for each word in the vocabulary.
            Shape: (batch_size, vocab_size)
        """

        # Pass input through embedding layer (output shape: (batch_size, context_window * 2, embed_dim))
        emb = self.embedding(x)

        # Compute the average of the embeddings (shape: (batch_size, embed_dim))
        average = torch.mean(emb, dim=1)

        # Normalize the averaged embedding (L2 normalization)
        normalized_average = average / average.norm(dim=1, keepdim=True)

        # Pass through the linear layer (shape: (batch_size, vocab_size))
        out = self.linear(normalized_average)

        return out


In [ ]:
from torch.utils.data import DataLoader

# Create DataLoader for training data
batch_size = 64  # Adjust this based on your GPU's capacity
train_dataloader = DataLoader(
    list(zip(surroundings, targets)), batch_size=batch_size, shuffle=True
)

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ensure 'word_to_index_mapping' is the correct dictionary from your vocabulary
vocab_size = len(word_to_index_mapping)  # Use the correct variable here

# Initialize the CBOW model, loss function, and optimizer
model = CBOW(vocab_size).to(device)  # Move model to device (GPU/CPU)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Create TensorDataset and DataLoader from surroundings and targets
train_dataset = TensorDataset(surroundings, targets)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training loop
epochs = 50
for epoch in range(epochs):
    total_loss = 0  # Initialize total loss for the epoch
    model.train()  # Set the model to training mode

    # Iterate over batches from DataLoader
    for surr, tar in tqdm(train_dataloader):
        # Move data to the appropriate device (GPU or CPU)
        surr, tar = surr.to(device), tar.to(device)

        # Clear gradients for the optimizer
        optimizer.zero_grad()

        # Forward pass: Get predictions from the model
        log_probs = model(surr)

        # Compute the loss between predicted output and actual target
        loss = loss_function(log_probs, tar)

        # Add the loss to the running total loss
        total_loss += loss.item()

        # Backward pass: compute gradients
        loss.backward()

        # Update the model parameters
        optimizer.step()

    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader)}")



100%|██████████| 117/117 [00:05<00:00, 20.76it/s]


Epoch 1/50, Loss: 7.618497192350208


100%|██████████| 117/117 [00:04<00:00, 25.17it/s]


Epoch 2/50, Loss: 7.305544930645543


100%|██████████| 117/117 [00:03<00:00, 37.06it/s]


...

Epoch 48/50, Loss: 1.4258655849685016


100%|██████████| 117/117 [00:02<00:00, 45.84it/s]


Epoch 49/50, Loss: 1.356891421171335


100%|██████████| 117/117 [00:03<00:00, 38.45it/s]

Epoch 50/50, Loss: 1.290861964225769


In [ ]:
import torch

def get_embedding(word, model, word_to_index_mapping):
    """
    Method to get the embedding vector for a given word.

    Arguments
    ---------
    word : String
        Word for which embedding is required.
    model : nn.Module
        Trained CBOW model containing the embedding layer.
    word_to_index_mapping : Dictionary
        Dictionary mapping words to index with format {word: index}.

    Returns
    -------
    word_embedding : Tensor
        Embedding vector for the given word.
    """
    # Get the index for the given word, or use <OOV> if word not found
    index = word_to_index_mapping.get(word, word_to_index_mapping.get('<OOV>'))

    # Ensure inference mode (no gradient calculation)
    with torch.no_grad():
        # Access the embedding layer weights
        embedding_weights = model.embedding.weight
        embedding_weights.requires_grad = False  # Ensure no gradient calculation

        # Extract the embedding vector for the given word index
        word_embedding = embedding_weights[index]

    return word_embedding

# Test the function with your dataset by getting the embedding of "capital"
word_embedding = get_embedding("capital", model, word_to_index_mapping)

# Print the resulting embedding
print(f"Embedding for 'capital': {word_embedding}")


Embedding for 'capital': tensor([ 8.2835e-01, -3.9596e-01, -1.2453e+00, -3.1056e-01, -9.2139e-01,
         1.4614e-01,  3.0492e-02,  3.4181e-01, -1.7110e+00,  1.2635e+00,
         5.2025e-01,  3.8489e-01,  8.3004e-01,  7.6644e-01,  1.2155e+00,
        -1.2502e+00,  3.2665e-01, -1.2754e+00, -3.5030e-01, -2.9372e-01,
        -1.4092e+00, -1.4293e+00,  7.5238e-01,  4.3461e-01, -1.6176e+00,
         3.6742e-01,  3.3168e-01,  3.5906e-01, -8.9272e-01, -2.9902e+00,
         6.7162e-01, -9.9007e-01,  5.6658e-01, -1.8158e+00,  8.5976e-01,
        -2.3137e-01, -2.0013e-01,  1.6528e+00, -4.9026e-01,  4.8632e-01,
        -1.2964e+00,  2.7427e-01, -1.3382e+00, -7.2405e-01, -1.1498e+00,
         2.0763e+00, -2.0316e-01,  3.1861e-01, -8.8944e-01, -1.4107e+00,
        -8.4982e-01, -1.2443e-01,  1.8189e+00, -1.4448e+00, -2.1442e+00,
         3.0062e-01,  8.0135e-01,  1.6293e+00,  4.7966e-02, -4.1063e-01,
         2.0211e+00, -8.8137e-01,  4.0413e-01, -3.6083e-01,  1.9096e-01,
         9.0848e-01,  8.39

In [ ]:
import torch

def cosine_similarity(v1, v2):
    """
    Method to calculate cosine similarity between two vectors.

    Arguments
    ---------
    v1 : Tensor
        First vector.
    v2 : Tensor
        Second vector.

    Returns
    -------
    cosine_sim : float
        Cosine similarity between v1 and v2.
    """
    dot_product = torch.dot(v1, v2)  # Calculate the dot product
    v1_norm = torch.norm(v1)  # Calculate the norm of v1
    v2_norm = torch.norm(v2)  # Calculate the norm of v2

    # Avoid division by zero
    if v1_norm == 0 or v2_norm == 0:
        return 0.0

    cosine_sim = dot_product / (v1_norm * v2_norm)  # Compute cosine similarity
    return cosine_sim.item()  # Convert to Python float

# Test the cosine similarity between two words using your CBOW model
def test_cosine_similarity(word1, word2, model, word_to_index_mapping):
    """
    Method to test cosine similarity between the embeddings of two words.

    Arguments
    ---------
    word1, word2 : String
        Words to compare.
    model : nn.Module
        Trained CBOW model.
    word_to_index_mapping : Dictionary
        Dictionary mapping words to indices.

    Returns
    -------
    similarity : float
        Cosine similarity between the embeddings of the two words.
    """
    # Get the embeddings for both words
    embedding1 = get_embedding(word1, model, word_to_index_mapping)
    embedding2 = get_embedding(word2, model, word_to_index_mapping)

    # Calculate and return the cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity

# Example usage
word1 = "capital"
word2 = "state"

similarity = test_cosine_similarity(word1, word2, model, word_to_index_mapping)
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity}")


Cosine similarity between 'capital' and 'state': 0.15685120224952698


In [ ]:
import torch

def get_k_nearest_words(k, word, vocabulary, model, word_to_index):
    """
    Method to find the k nearest words of a given word.

    Arguments
    ---------
    k : int
        Number of nearest words to return.
    word : String
        Word for which to find the k nearest words.
    vocabulary : Dictionary
        Dictionary mapping words to their frequency (or presence) {word: frequency}.
    model : nn.Module
        Trained CBOW model.
    word_to_index : Dictionary
        Dictionary mapping words to index {word: index}.

    Returns
    -------
    similar : List of Strings
        List of k nearest words to the given word.
    """

    # Get the embedding for the given word using the model
    word_embedding = get_embedding(word, model, word_to_index)

    # Initialize a tensor to store similarity scores (1 score per word in the vocabulary)
    similarity_scores = torch.zeros(len(vocabulary))

    # Get a list of vocabulary keys once (for efficiency)
    vocab_keys = list(vocabulary.keys())

    # Iterate over all words in the vocabulary to compute similarities
    for i, other_word in enumerate(vocab_keys):
        # Skip if the word is the same as the input word
        if other_word == word:
            similarity_scores[i] = -float('inf')  # Assign a very low score to skip
            continue

        # Get the embedding of the other word
        other_word_embedding = get_embedding(other_word, model, word_to_index)

        # Compute cosine similarity between the word and the other word
        similarity_scores[i] = cosine_similarity(word_embedding, other_word_embedding)

    # Get the indices of the top k most similar words using torch.topk
    k_first = torch.topk(similarity_scores, k)

    # Extract the indices of the k most similar words
    similar_word_indices = k_first.indices

    # Convert the indices back to words from the vocabulary
    similar = [vocab_keys[index] for index in similar_word_indices]

    return similar

# Example usage:
similar_words = get_k_nearest_words(5, "capital", vocabulary, model, word_to_index_mapping)
print(f"Top 5 words similar to 'capital': {similar_words}")


Top 5 words similar to 'capital': ['seat', 'houses', 'end', 'rest', 'cordilleran']


In [ ]:
import pandas as pd
import torch

def test_analogy(model, word_to_index_mapping, analogy_file, subset_size=None):
    """
    Method to test accuracy of CBOW embeddings on analogy tasks.

    Arguments
    ---------
    model : nn.Module
        Trained CBOW model.
    word_to_index_mapping : Dictionary
        Dictionary mapping words to indices {word: index}.
    analogy_file : String
        File containing analogy tasks.
    subset_size : int, optional
        Number of rows to use from the dataset (for testing purposes).

    Returns
    -------
    accuracy : float
        Accuracy of the model on the analogy tasks.
    """
    # Load the CSV file
    df = pd.read_csv(analogy_file)

    # Optional: Reduce dataset size for quick testing
    if subset_size:
        df = df.head(subset_size)

    correct = 0
    total = 0
    skipped = 0  # Counter for skipped tasks

    # Iterate through each analogy task
    for index, row in df.iterrows():
        word_one = row['word_one'].lower()
        word_two = row['word_two'].lower()
        word_three = row['word_three'].lower()
        word_four = row['word_four'].lower()

        # Skip tasks if any word is not in the vocabulary
        if (word_one not in word_to_index_mapping) or \
           (word_two not in word_to_index_mapping) or \
           (word_three not in word_to_index_mapping) or \
           (word_four not in word_to_index_mapping):
            skipped += 1
            continue

        # Get embeddings for the words
        embedding_word_one = get_embedding(word_one, model, word_to_index_mapping)
        embedding_word_two = get_embedding(word_two, model, word_to_index_mapping)
        embedding_word_three = get_embedding(word_three, model, word_to_index_mapping)

        # Compute the analogy vector
        analogy_vector = embedding_word_two - embedding_word_one + embedding_word_three

        # Get the top 3 nearest words to the analogy vector
        predictions = get_k_nearest_words(3, word_three, vocabulary, model, word_to_index_mapping)

        # Check if the correct word is among the predictions
        is_correct = word_four in predictions
        print(f"Analogy: {word_one} -> {word_two} :: {word_three} -> {word_four} | "
              f"Prediction: {predictions}, Correct: {is_correct}")

        if is_correct:
            correct += 1

        total += 1

    # Print summary stats
    print(f"Total tasks skipped due to missing words: {skipped}")
    if total == 0:
        print("No valid tasks were processed.")
        return 'No word was found in the embeddings'

    accuracy = correct / total
    print(f"Analogy task accuracy: {accuracy:.4f}")
    return accuracy

# Ensure the word-to-index mapping is generated from your vocabulary
word_to_index_mapping = word_to_index(vocabulary)  # Call function to generate the dictionary

# Test the model on your uploaded dataset using the correct dictionary
accuracy = test_analogy(model, word_to_index_mapping, 'us_states_capitals_analogy.csv')



Analogy: alabama -> montgomery :: hawaii -> honolulu | Prediction: ['states', 'juan', 'intermontane'], Correct: False
Analogy: alabama -> montgomery :: massachusetts -> boston | Prediction: ['greater', 'highest', 'boston'], Correct: True
Analogy: alabama -> montgomery :: texas -> austin | Prediction: ['deep', 'brown', 'allow'], Correct: False
Analogy: alabama -> montgomery :: vermont -> montpelier | Prediction: ['supreme', 'seismic', 'india'], Correct: False
Analogy: alabama -> montgomery :: virginia -> richmond | Prediction: ['snowfall', 'lush', 'used'], Correct: False
Analogy: hawaii -> honolulu :: massachusetts -> boston | Prediction: ['greater', 'highest', 'boston'], Correct: True
Analogy: hawaii -> honolulu :: texas -> austin | Prediction: ['deep', 'brown', 'allow'], Correct: False
Analogy: hawaii -> honolulu :: vermont -> montpelier | Prediction: ['supreme', 'seismic', 'india'], Correct: False
Analogy: hawaii -> honolulu :: virginia -> richmond | Prediction: ['snowfall', 'lush', 